# Isolation Forests

> Objective
> * Implement the Isolation Forest (iForest) anomaly detection algorithm

Original ["Isolation Forest"](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf) paper

### Importance to project

Compared to all previous methods, the isolation forest method provides a very different approach to finding anomalies. Instead of explicitly relying on statistics such as mean or covariance, or probabilities such as the cumulative distribution function, it is based upon the concepts of decision trees and ensemble methods widely used in the supervised machine learning literature. In this project, I will implement the Isolation forest anomaly detection method.

## Isolation forest in a nutshell

Isolation forests exploit the heuristic that anomalies tend to be isolated points. That is, points that are separated from the rest either because they are characterized by extreme values along some feature dimension or because they are in the region of rather low density. Instead of modeling these distance or density metrics like, for example, PCA or ECOD, the method seeks to directly isolate anomalous points.

The method isolates points by repeatedly partitioning the data. The partition criteria are randomly built at every iteration; first, a feature for analysis is randomly selected, then for that feature, its smaller and larger values are found, and then a `split value` is generated by drawing a uniform random number between the feature’s min and max value. All observations whose analyzed features have a value less than the `split value` are gathered into one group; the reminder observations are gathered in a separate group. In turn, each of the subgroups will be partitioned in a similar fashion. At each step, the information of what feature and split value was used is recorded in a node together with the children nodes that will contain similar information for each of the subgroups. The process is carried out until the isolation tree reaches a maximum height of $\log_2(\psi)$ where $\psi$ is the number of sub-samples used to build the tree and is a parameter of the algorithm. When the maximum tree height is reached, the leaf nodes only store the information of how many samples were allocated to it.

For example, in the figure below, the data is first split according to the values of feature 5, it is found that only a single observation has a feature 5 value larger than the split value of -3, so that observation is directly isolated. The remaining observations are further split according to the value of feature 3 and at later stages features 1 and 5. At that point the maximum height of the tree is reached and the leaf nodes only contain information about the number of samples allocated to them.

The intuition of the algorithm, is that anomalous endpoints tend to be closer to the root of the tree. This notion of closeness is formalized by the `path length` from the root to the isolating node i.e. the number of steps taken to reach the node from the root of the tree. The shorter the path length the more anomalous an endpoint is. Notice that any sample can always be scored even if it was not used to build the tree, it is enough to apply the partition criteria at every node until a leaf node is reached.

Using a single tree to score all observations would lead to results with high variance, this is why the algorithm builds a predefined number of trees and generates the anomaly score by averaging the path length over all the trees. The final anomaly anomaly score is given by the formula.

$$S(x) = 2^{-E(x)/C(n)}$$

$E(x)$ is the actual path length average of sample $x$ over all trees and $C(n)$ is a normalization factor. Technically, $C(n)$ calculates the average path length of an unsuccessful search on a binary search tree with $n$ nodes. The reason for using it is that it allows to compare results that might have been generated by trees of different shapes.

The method has two parameters, subsample size and the number of trees, in the unsupervised setting this is usually a drawback because of the difficulty of tuning parameters. In the case of isolation forests, it turns out that the algorithm is rather insensitive to the values of these parameter and that values around of 100 to 1000 trees and sub sample sizes of 256 to 1000 tend to deliver similar results across multiple data sets.

There are many technical details to each of the steps needed to implement the isolation fores method, but don’t worry you will work on them one step at a time.
  
<br>   
<div align="center">
<img src="https://d16rtcb5cr0vb4.cloudfront.net/C1615+Advanced+Multidimensional+Anomaly+Detection+with+Isolation+Forests%2FResources%2FImages%2FP4_ISOforest_for+approval_V1.png?Expires=1691070962&Signature=E8OGgSgeWGtJRARbW372UYICHgZFHbHJObZDMor6Dzdt6z7cLe6LDAUo3kVLI7aE11tGN6TXy87DFCyv9u6h2tuiBq5C5WxrdvrHrLcq3vahj7XP3GZ02wtAk0mpCm80EX4U6nh8qIUKojkfYp6msaEyUIVPRhIUUVTWzNb4S6lS0sceu080U9PLo1lV~ra8mx3aU8DYCex4SSe2uYgX2SXVabAU~cwtoEvC0h2qvNTk0vsR~iZodQguNDi3BD8ZSHrsHaoRGlo0uZ9DZ6PsPwTGAU04~P0MBFSLxblrONx5o7LJ2ucmaY~-n1TpHaMgG6p3tur4Cy7E7oUG1mdoEA__&Key-Pair-Id=APKAIHLKH2FX732Z3HGA" alt="drawing" width="35%"/>
</div>

## Workflow

The isolation forest method is an ensemble method. This means it combines the predictions made by multiple simple components into one improved result. In this case, the simple components are isolation trees. Technically, an isolation tree is a binary tree where each internal node contains a feature index, a split value, and two children nodes. External or leaf nodes contain only the number of observations that were assigned to it at training time.

The approach to implement the Isolation forest is going to be:

* A) Define the tree data structures  
* B) Implement the tree-building function  
* C) Implement the forest building function  
* D) Implement the scoring function  
* E) Assemble all the pieces into the `IsoForestScorer` class  
  
In the starter file, you will find the stubs for the relevant functions that will be implemented. Additionally, you will find various tests that should help you while developing.  
To run them, use `pytest pytest -v tests/milestone4`

## A) Tree data structures
The isolation trees data structures are:

```python
class Node:
    pass

@dataclass()
class ExNode(Node):
    size: int

@dataclass()
class InNode(Node):
    left: Node
    right: Node
    splitAtt: int
    splitValue: float
```

The `Node` class has two sub-classes:

`ExNode`: That describes the outermost nodes when no further partitions are made  
`INode`: That describes the inner nodes of the tree.

## B) Tree building
The algorithm for building a single tree is

`iTree(X,e,l)`

**Inputs:** observations $X$ ($n$ observations each with $d$ features), current tree height $e$ and max tree hight $l$.

**Output:** an isolation tree

`if` $e≥l$ `or` $∣ X ∣≤1$ `then`

`return ExternalNode(size = |X|)`

`else`

split_attribute <- randomly select the index of a feature

split_value <- uniform random number between the min and max value of all values of feature `split_attribute` in $X$

left_data <- all samples in $X$ such that their `split_attribute` feature is smaller than `split_value`

right_data <- all samples in $X$ such that their `split_attribute` feature is larger or equal than `split_value`

left_node = `iTree(left_data,e+1,l)`

right_node = `iTree(right_data,e+1,l)`

`return InternalNode(left_node,right_node,split_attribute,split_value)`

It can be implemented in the following way:

1. Implement the `split_data(data, feature_index, split_point)` function that takes a set of observations, a feature index and a split point and returns left_data and right_data

2. Implement the `calculate_split_point(data, feature_index, rng)` function that takes a set of observations, a feature index and a random number generator. It returns a random number uniformly sampled between the min and max values of the feature corresponding to feature_index in all of data

3. Implement the `build_tree(data, current_height, max_height, rng)` function, that corresponds to `iTree(X,e,l)`, using the `split_data` and `calculate_split_point` functions.

<br>
<div align="center">
    <img src='img/iTree_algo.png' alt='iTree alogirthm' width="30%"/>
</div>



In [51]:
#Let's start with utils
import numpy as np
import random
import scipy.io
from dataclasses import dataclass

def matlab_to_numpy(path_to_mat_file: str)-> np.ndarray:
    """Convert MATLAB .mat file to Numpy array

    Args:
        path_to_mat_file (str): _description_

    Returns:
        np.ndarray: _description_
    """
    data=scipy.io.loadmat(path_to_mat_file)
    data=data['X']
    
    return np.array(data)

class Node:
    pass

@dataclass()
class ExNode(Node):
    size: int

@dataclass()
class InNode(Node):
    left: Node
    right: Node
    splitAtt: int
    splitValue: float

In [54]:
# Let's start with split_data function


def split_data(data: np.ndarray, 
               feature_index: int, 
               split_point: int):
    """Takes a set of observations, a feature index and a 
    split point and returns left_data and right_data

    Args:
        data (_type_): _description_ MATLAB matrix composed of X and y dict
        feature_index (_type_): _description_
        split_point (_type_): _description_

    Returns:
        _type_: _description_
    """
    left_data = data[:split_point, feature_index]
    right_data = data[split_point:, feature_index]
    
    return (left_data, right_data)

def calculate_split_point(data: dict, 
                          feature_index: int, 
                          rng: np.random._generator.Generator) -> int:
    """Takes a set of observations, a feature index and a random number 
    generator. It returns a random number uniformly sampled between the 
    min and max values of the feature corresponding to feature_index in 
    all of data

    Args:
        data (dict): _description_
        feature_index (int): _description_
        rng (int): _description_

    Returns:
        int: _description_
    """
    min_feat = min(data[:, feature_index])
    max_feat = max(data[:, feature_index])
    rnd_nbr = rng.integers(min_feat, max_feat)
    
    return rnd_nbr

def build_tree(data: dict, 
               current_height: int, 
               max_height: int, 
               rng: np.random._generator.Generator):
    """_summary_

    Args:
        data (dict): _description_
        current_height (int): _description_
        max_height (int): _description_
        rng (None): _description_

    Returns:
        _type_: _description_
    """
    dim_n, dim_d = np.shape(data)
    
    if current_height>=max_height or dim_n<=1:
     
        return ExNode(size=dim_n)
    
    else:
        feature_index = rng.integers(dim_d)
        split_point = calculate_split_point(data, feature_index, rng)
        left_data, right_data = split_data(data, 
                                           feature_index, 
                                           split_point)
        left_node = build_tree(left_data, 
                               current_height + 1, 
                               max_height, 
                               rng)
        right_node = build_tree(right_data, 
                                current_height + 1, 
                                max_height, 
                                rng)

        return InNode(left_node, right_node, feature_index, split_point)

In [58]:
#TEST
test_data = matlab_to_numpy('datasets/cover.mat')

# 1st function
left, right = split_data(test_data, 3, 34)
print(left[:10])
print(right[:10])

# 2nd function
rng_test = np.random.default_rng(12345)

split_point_test = calculate_split_point(test_data, 3, rng_test)
print(split_point_test)

[268 242 300 371 150 216 323 212 242   0]
[426  85 543   0 190 351 150 150 170 162]
976


## C) Forest building
An isolation forest is just a collection of isolation trees. The algorithm is:

`IForest(X,t,s)`

**Inputs:** observations $X$ ($n$ observations each with $d$ features), number of tree $t$ sub sample size $s$  
**Output:** Isolation forest, list of isolation trees.

`initialize Forest` (list that will contain the trees)

max_height <- $ceiling(log_2(s))$

`for` i = 1 to t `do` 

$X'$  <- sample without replacement $s$ samples from $X$

iTree <- `iTree(X',0,max_height)`

append `iTree` to `Forest`

`return Forest`

1. Implement `IForest(X,t,s)`

In [61]:
def IForest(data: dict, t: int, s: int):
    rng = np.random.default_rng(12345)
    max_height = np.ceil(np.log(s))
    forest = []
    for _ in range(t):
        sampled_data = rng.choice(data, s, replace=False)
        tree = build_tree(sampled_data, 0, max_height, rng)
        forest.append(tree)

    return forest

## D) Scoring function
To calculate the anomaly score for a given sample, it is required to first calculate the path length of that sample on all trees and second to average and normalize the result.

### Path length algorithm:

`PathLength(x,T,e)`

**Inputs:** $x$ single observation, $T$ an isolation tree, $e$ current path length  
**Output:** path length of $x$ `if` $T$ is an external node `then` return $e + c_n(T.size)$ `end if`

a <- T.split_attribute

x_a <- a-th feature of $x$

if x_a < T.split_value then return PathLength(x,T.left,e + 1) else return PathLength(x,T.right,e + 1)

$c_n(.)$ is a function that returns the average path length of an unsuccessful search on a binary search tree.

### Score sample algorithm:

`ScoreSample(x,F,s)`

**Inputs:** observation $x$, isolation forest $F$ sub sample size $s$  
**Output:** Outlier score of $x$

initialize PL (list that will contain the path scores)

for each tree T in F path_length <- PathLength(x,T,e) append path_length to PL

mean_path_length =mean of PL values

$score <- 2^{\frac{mean \space path \space length}{c_n(s)}}$  

return score

1. Implement the path length algorithm; see the `path_length` function in the starter file
2. Implement the score sample algorithm; see the `score_sample` function in the starter file

In [62]:
def c_n(n):
    """
    Args:
        n: number of nodes in a binary search tree

    Returns
        average path length of an unsuccessful search on the try
    """
    if n > 2:
        h_n_1 = np.log(n - 1) + np.euler_gamma
        return 2 * h_n_1 - (2.0 * (n - 1) / n)
    elif n == 2:
        return 1
    else:
        return 0

def path_length(sample, 
                tree, 
                current_path_length):

    match tree:
        case ExNode(size):
            return current_path_length + c_n(size)
        case InNode(left, right, splitAtt, splitValue):
            if sample[splitAtt] < splitValue:
                return path_length(sample, 
                                   left, 
                                   current_path_length + 1)
            else:
                return path_length(sample, 
                                   right, 
                                   current_path_length + 1)


def score_sample(sample, 
                 forest, 
                 num_samples):
    """
    Args:
        sample: single observation i.e instance to be scored
        forest: list of trees
        num_samples: number of samples used to build the trees in `forest`

    Returns
        outlier score of `sample`
    """
    path_len = []
    for tree in forest:
        sample_path_len = path_length(sample, tree, 0)
        path_len.append(sample_path_len)

    mean_path_length = np.mean(path_len)
    exp = mean_path_length / c_n(num_samples)
    score = np.float_power(2.0, -exp)
    
    return score

## E) IsoForestScorer
Bring all the pieces together and implement the `IsoForestScorer` class

1. Implement the initialization logic __init__.
    * what data should be kept for the score function to work?
    
2. Implement the score functionality

In [ ]:
class IsoForestScorer:

    def __init__(self, data, num_trees, sub_sample_size, rng) -> None:

        self.sub_sample_size = sub_sample_size
        self.forest = IForest(data, num_trees, sub_sample_size, rng)

    def score(self, samples):

        def _scorer(sample):
            return score_sample(sample, self.forest, self.sub_sample_size)

        return np.apply_along_axis(_scorer, 1, samples)
    

## F) Final check
The isolation forest algorithm is certainly not trivial; by this point, you should have a working version of it.

1. Run the starter file and verify that your implementation produces the same results as those presented in the original publication and in the implementation provided by scikit learn.

**Note:** the

* mammography (http://odds.cs.stonybrook.edu/mammography-dataset/)
* breast (http://odds.cs.stonybrook.edu/mammography-dataset/)
* shuttle (http://odds.cs.stonybrook.edu/shuttle-dataset/)
  
datasets must be available on the `datasets` folder of the project to run the final check.